In [1]:
import pandas as pd
import numpy as np

import pickle

In [34]:
preds_path_ls = [
    ('raw', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\lr_search\run_raw.bin'),
    ('cfg11', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\val_preds\cfg11.bin'),
    ('cfg12', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\val_preds\cfg12.bin'),
    ('cfg13', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\val_preds\cfg13.bin'),
]

In [35]:
preds_dfs = {}

for run_name, preds_path in preds_path_ls:
    with open(preds_path, 'rb') as f:
        preds_dfs[run_name] = pickle.load(file=f)

In [36]:
preds_dfs['cfg11'].head()

,generated_text,subject,pred,true,corr,category
0,D. 6,abstract_algebra,D,A,0,STEM
1,C. 6,abstract_algebra,C,B,0,STEM
2,"A. True,",abstract_algebra,A,A,1,STEM
3,D. 12,abstract_algebra,D,C,0,STEM
4,D. 24,abstract_algebra,D,C,0,STEM


In [37]:
acc_by_subjects = pd.DataFrame({
    'subject': list(set(preds_dfs['cfg11'].subject))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['subject', 'corr']].groupby(['subject'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_subjects = acc_by_subjects.merge(
        acc_df,
        left_on='subject',
        right_on='subject',
        how='right'
    )

In [38]:
acc_by_subjects

,subject,raw,cfg11,cfg12,cfg13
0,abstract_algebra,0.272727,0.090909,0.090909,0.363636
1,anatomy,0.500000,0.571429,0.571429,0.571429
2,astronomy,0.500000,0.687500,0.812500,0.750000
3,business_ethics,0.909091,0.636364,0.636364,0.545455
4,clinical_knowledge,0.655172,0.689655,0.724138,0.758621
5,college_biology,0.750000,0.750000,0.750000,0.750000
6,college_chemistry,0.500000,0.250000,0.250000,0.250000
7,college_computer_science,0.363636,0.545455,0.454545,0.545455
8,college_mathematics,0.454545,0.272727,0.363636,0.363636
9,college_medicine,0.636364,0.681818,0.636364,0.636364


In [39]:
acc_by_categories = pd.DataFrame({
    'category': list(set(preds_dfs['cfg11'].category))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['category', 'corr']].groupby(['category'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_categories = acc_by_categories.merge(
        acc_df,
        left_on='category',
        right_on='category',
        how='right'
    )

In [40]:
acc_by_categories

,category,raw,cfg11,cfg12,cfg13
0,STEM,0.504673,0.445483,0.467290,0.495327
1,humanities,0.527027,0.546332,0.548263,0.575290
2,"other (business, health, misc.)",0.695775,0.721127,0.707042,0.740845
3,social sciences,0.732938,0.732938,0.738872,0.756677


In [5]:
preds_df[['category', 'corr']].groupby(['category']).mean()

,corr
category,
STEM,0.525182
humanities,0.527099
"other (business, health, misc.)",0.706663
social sciences,0.721807


In [6]:
total_accuracy = preds_df['corr'].mean()
total_correctness = (preds_df['pred'] != 'I').mean()

print(f"{total_accuracy=}")
print(f"{total_correctness=}")

total_accuracy=0.6108104258652614
total_correctness=0.997151402934055
